<a href="https://colab.research.google.com/github/Hypokeimenon/desafio_bootcamp/blob/main/super_desafio_pietro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análise de correlação entre deficiência de Vitamina D e o agravamento do quadro clínico de COVID19 - por Pietro Sanchini

#Objetivo

O objetivo deste estudo é analisar as possíveis relações de correlação e causalidade existentes entre a ingestão regular de vitamina D através da exposição à luz solar ou através de alimentos, e a evolução da sintomatologia de pessoas que contraíram COVID19

#Metodologia

O estudo foi feito cruzando informações de duas bases distintas, sendo uma com as informações sobre alimentação e região (rural ou urbana) de moradores do Brasil e a segunda base, do Brasil.IO, com os dados de mortes e casos confirmados da COVID19. Estes dados foram separados, quando possível, por estados, entendendo que Pernambuco é um estado preponderantemente quente, com maior incidência de luz solar, e Santa Catarina é um estado com maior incidência de períodos frios e possível índice aumentado de confinamento em períodos de baixa temperatura.

#Iníciando a codificação

Importação das bibliotecas

In [202]:
import pandas as pd
import numpy as np

Acessando o drive do Google

In [203]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Tratando e limpando a base a respeito de hábitos alimentares

Abertura da base de hábitos alimentares

In [204]:
alimentacao_simples = pd.read_csv("/content/drive/MyDrive/datasets/all_cnty_ac_yr_2015.csv")
alimentacao_simples.head()

,superregion2,iso3,countryname,female,adult,urban,edu,v01_wt_median,v02_wt_median,v05_wt_median,v06_wt_median,v10_wt_median,v15_wt_median,v16_wt_median,v23_wt_median,v36_wt_median,v41_wt_median,v57_wt_median
0,SAARC,AFG,Afghanistan,0,0,0,1,60.40,127.06,52.87,4.61,28.07,251.30,20.67,58.11,456.28,2020.18,104.62
1,SAARC,AFG,Afghanistan,0,0,0,2,75.33,135.78,57.84,5.70,33.45,282.21,29.70,59.89,478.07,2093.73,143.43
2,SAARC,AFG,Afghanistan,0,0,0,3,89.39,151.52,63.04,6.71,39.64,249.74,37.25,60.38,507.93,2183.09,175.80
3,SAARC,AFG,Afghanistan,0,0,0,999,62.22,128.49,53.50,4.74,28.78,252.00,21.72,58.26,459.39,2030.11,109.12
4,SAARC,AFG,Afghanistan,0,0,1,1,70.64,133.13,57.43,4.88,33.87,205.35,27.41,59.21,483.85,1952.97,124.06


In [205]:
alimentacao_simples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   superregion2   19980 non-null  object 
 1   iso3           19980 non-null  object 
 2   countryname    19980 non-null  object 
 3   female         19980 non-null  int64  
 4   adult          19980 non-null  int64  
 5   urban          19980 non-null  int64  
 6   edu            19980 non-null  int64  
 7   v01_wt_median  19980 non-null  float64
 8   v02_wt_median  19980 non-null  float64
 9   v05_wt_median  19980 non-null  float64
 10  v06_wt_median  19980 non-null  float64
 11  v10_wt_median  19980 non-null  float64
 12  v15_wt_median  19980 non-null  float64
 13  v16_wt_median  19980 non-null  float64
 14  v23_wt_median  19980 non-null  float64
 15  v36_wt_median  19980 non-null  float64
 16  v41_wt_median  19980 non-null  float64
 17  v57_wt_median  19980 non-null  float64
dtypes: flo

In [206]:
aldf = alimentacao_simples.query("iso3 in ['BRA']") #localizando os dados referentes ao Brasil
aldf = aldf.rename(columns={'iso3':'country', 'female':'genre', 'urban':'region', 'adult':'age_range', 'v01_wt_median':'fruit', 'v02_wt_median':'Non_starchy_vegetables', 'v05_wt_median':'Beans_and_legumes', 'v06_wt_median':'Nuts_and_seeds', 'v10_wt_median':'Unprocessed_red_meats', 'v15_wt_median':'Sugar-sweetened_beverages', 'v16_wt_median':'Fruit_juices', 'v57_wt_median':'Total_milk', 'v23_wt_median':'Total_protein'})

In [207]:
aldftype = {'region': 'string', 'genre': 'string', 'age_range':'string'} #alterando o tipo de dados da coluna
aldf = aldf.astype(aldftype)

In [208]:
aldf.replace({'region': {'0':'rural', '1':'urban', '999':'others'}}, inplace=True) #melhorando a visualização das informações 
aldf.replace({'genre': {'0':'male', '1':'female', '999':'both'}}, inplace=True)
aldf.replace({'age_range': {'0':'young', '1':'adult', '999':'all_ages'}}, inplace=True)

In [209]:
aldf.drop(['superregion2', 'v36_wt_median', 'v41_wt_median', 'edu', 'countryname'], axis='columns', inplace=True) #dropando as colunas que não usarei
aldf.head()

,country,genre,age_range,region,fruit,Non_starchy_vegetables,Beans_and_legumes,Nuts_and_seeds,Unprocessed_red_meats,Sugar-sweetened_beverages,Fruit_juices,Total_protein,Total_milk
2376,BRA,male,young,rural,80.16,76.48,78.43,0.24,44.91,155.00,41.16,71.54,75.23
2377,BRA,male,young,rural,94.47,85.87,80.75,0.29,55.42,159.16,56.28,73.40,106.26
2378,BRA,male,young,rural,105.27,102.46,83.48,0.38,61.44,147.01,72.98,74.78,137.68
2379,BRA,male,young,rural,83.84,79.64,79.11,0.26,47.46,154.86,45.46,72.02,83.83
2380,BRA,male,young,urban,90.51,79.03,74.76,0.38,53.64,180.40,60.86,78.33,93.48


#Tratando e limpando a base a respeito do COVID19

Abertura da base sobre o COVID19 no Brasil

In [210]:
covid = pd.read_csv("/content/drive/MyDrive/datasets/caso.csv")
covid.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-11-05,AC,NaN,state,31248,697,234,True,881935.00,894470.00,12.00,3493.47,0.02
1,2020-11-04,AC,NaN,state,31218,696,233,False,881935.00,894470.00,12.00,3490.11,0.02
2,2020-11-03,AC,NaN,state,30962,695,232,False,881935.00,894470.00,12.00,3461.49,0.02
3,2020-11-02,AC,NaN,state,30954,693,231,False,881935.00,894470.00,12.00,3460.60,0.02
4,2020-11-01,AC,NaN,state,30951,693,230,False,881935.00,894470.00,12.00,3460.26,0.02


In [211]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936870 entries, 0 to 936869
Data columns (total 13 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   date                            936870 non-null  object 
 1   state                           936870 non-null  object 
 2   city                            930535 non-null  object 
 3   place_type                      936870 non-null  object 
 4   confirmed                       936870 non-null  int64  
 5   deaths                          936870 non-null  int64  
 6   order_for_place                 936870 non-null  int64  
 7   is_last                         936870 non-null  bool   
 8   estimated_population_2019       933215 non-null  float64
 9   estimated_population            933215 non-null  float64
 10  city_ibge_code                  933215 non-null  float64
 11  confirmed_per_100k_inhabitants  919888 non-null  float64
 12  death_rate      

In [212]:
covdf = covid.query("state in ['PE', 'SC']") #selecionando os Estados que serão analisados

In [213]:
covdf.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
335524,2020-11-05,PE,NaN,state,164222,8687,239,True,9557071.00,9616621.00,26.00,1707.69,0.05
335525,2020-11-04,PE,NaN,state,163642,8667,238,False,9557071.00,9616621.00,26.00,1701.66,0.05
335526,2020-11-03,PE,NaN,state,163156,8643,237,False,9557071.00,9616621.00,26.00,1696.60,0.05
335527,2020-11-02,PE,NaN,state,163039,8638,236,False,9557071.00,9616621.00,26.00,1695.39,0.05
335528,2020-11-01,PE,NaN,state,162977,8632,235,False,9557071.00,9616621.00,26.00,1694.74,0.05


In [214]:
covdf.drop(['city', 'place_type', 'order_for_place', 'is_last', 'estimated_population_2019', 'city_ibge_code'], axis='columns', inplace=True) #dropando as colunas que não utilizarei
covdf.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,state,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate
335524,2020-11-05,PE,164222,8687,9616621.00,1707.69,0.05
335525,2020-11-04,PE,163642,8667,9616621.00,1701.66,0.05
335526,2020-11-03,PE,163156,8643,9616621.00,1696.60,0.05
335527,2020-11-02,PE,163039,8638,9616621.00,1695.39,0.05
335528,2020-11-01,PE,162977,8632,9616621.00,1694.74,0.05


In [215]:
covdf['date'] = pd.to_datetime(covdf['date']) #alterando o tipo de dado da coluna para data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [216]:
covdf['month'] = covdf['date'].dt.month #trabalhando com meses
covdf.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,state,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,month
335524,2020-11-05,PE,164222,8687,9616621.00,1707.69,0.05,11
335525,2020-11-04,PE,163642,8667,9616621.00,1701.66,0.05,11
335526,2020-11-03,PE,163156,8643,9616621.00,1696.60,0.05,11
335527,2020-11-02,PE,163039,8638,9616621.00,1695.39,0.05,11
335528,2020-11-01,PE,162977,8632,9616621.00,1694.74,0.05,11


In [217]:
    covdf.isnull() #verificando se existem dados faltantes
    

,date,state,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,month
335524,False,False,False,False,False,False,False,False
335525,False,False,False,False,False,False,False,False
335526,False,False,False,False,False,False,False,False
335527,False,False,False,False,False,False,False,False
335528,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
721797,False,False,False,False,False,False,False,False
721798,False,False,False,False,False,False,False,False
721799,False,False,False,False,False,False,False,False
721800,False,False,False,False,False,False,False,False


#Análise

agrupamento das alimentações mais ricos em vitamina D por região para entender se existe diferenças de alimentação marcantes entre meio urbano e rural.

In [218]:
aldf_brasil = aldf.groupby('region')[['Unprocessed_red_meats', 'Total_protein', 'Total_milk']].sum() #
aldf_brasil.head()

,Unprocessed_red_meats,Total_protein,Total_milk
region,,,
others,2351.14,2990.28,3661.49
rural,2015.10,2764.50,3029.70
urban,2406.89,3027.74,3766.31


Agrupamento do número de mortes e pessoas contaminadas por mês para verificar evolução dos casos desde o início. Foi percebido, no entanto, que cada resultado replicava o valor anterior com a inclusão dos casos relacionados àquele dia, o que adulterava a amostragem com números astronômicos e que não refletiam a realidade.

In [219]:
covdf_estado = covdf.groupby('state')[['confirmed', 'deaths']].sum()
covdf_estado.head()

,confirmed,deaths
state,,
PE,34586008,2187626
SC,40554670,531134


Para contornar o erro anterior, foi utilizado o .loc para puxar os dados do último mês, com números totais de mortos e confirmados do último registro, retirando as demais colunas para melhor apresentação do conteúdo.

In [220]:
covdf_mortes_e_confirmados = covdf.loc[[335524, 668623]] #selecionando as datas mais recentes para acessar os números de mortes e casos confirmados do Estado de Pernambuco e Santa Catarina
covdf_mortes_e_confirmados

,date,state,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,month
335524,2020-11-05,PE,164222,8687,9616621.00,1707.69,0.05,11
668623,2020-11-05,SC,268644,3173,7252502.00,3704.16,0.01,11


In [221]:
covdf_mortes_e_confirmados.drop(['estimated_population', 'confirmed_per_100k_inhabitants', 'death_rate', 'month'], axis='columns', inplace=True) #dropando colunas que não utilizarei
covdf_mortes_e_confirmados

,date,state,confirmed,deaths
335524,2020-11-05,PE,164222,8687
668623,2020-11-05,SC,268644,3173


#Conclusão

Os dados anteriormente apresentados mostram que no Brasil há um maior número de mortos em Pernambuco comparado com Santa Catarina, apesar da relação ser inversa  quando falamos de casos confirmados. Também constatamos, com base na amostragem coletada da base de dados global sobre alimentação, que alimentos com maiores fontes de vitamina D, preponderantemente de origem animal, são consumidos em maior quantidade pela população em meio urbano; é importante ressaltar que, de acordo com fontes médicas pesquisadas, esse tipo de vitamina é mais presente em alimentos de origem animal como ovos, leite ou carnes.

Dado que este estudo iniciou com o interesse de entender se haviam variáveis que gerassem uma narrativa causal entre a maior ingestão de vitamina D e a menor gravidade de sintomas, um marcador importante foi ter observado o número de mortes por estado, já que muitas vezes o óbito pode indicar maior quantidade de pacientes que evoluíram para estados graves de saúde até quando acabassem perdendo a própria vida. Contudo, isso não pode ser uma relação post hoc, já que não necessariamente pessoas confirmadas deixam de abarcar casos graves que, no entanto, não vieram a falecer.

A princípio podemos deduzir que a população rural, provavelmente com costumes de menor confinamento em casas e edifícios fechados durante grande número de horas poderiam ter maior índice de vitamina D e consequentemente menor nível de óbitos ou de casos graves, com a relação inversa ocorrendo em grandes centros urbanos. O estudo induz através da observação dos dados, que talvez não exista esta relação já que também podem existir outras variáveis como suplementação e ingestão de alimentos que podem impactar na análise desta causalidade.

Existem algumas limitações no estudo, como a falta de amostragem contundente quando falamos de alimentação, já que temos a importação de uma base global, da Global Dietary Database (GDD)  que, apesar de muito robusta e gerada em colaboração com grandes instituições de ensino globais, traz apenas 108 registros no Brasil e sem segmentação por estado. Outras limitações são, conforme já dito anteriormente, a existência de fatores complexos e intercambiáveis que podem influenciar na análise e adulterar a possível relação entre variáveis.

Esta pesquisa sugere a necessidade de um maior aprofundamento posterior, com adição de maiores bases de dados de institutos locais como IBGE a fim de trazer outros fenômenos estruturados em rede como desenvolvimento sócio-econômico de cada estado, taxa de urbanização, renda per capita, taxa de escolarização e políticas públicas(taxas) de investimento em saúde. Com isso será possível ter mais contundência entendendo variáveis múltiplas, já que o maior número de mortes pode refletir, invés da deficiência de vitamina D, apenas um baixo investimento regional em saúde pública. Desta forma será possível dirimir possíveis ocorrências de vieses de ancoragem e falácia non sequitur.

#Fontes


- https://www.ibge.gov.br/cidades-e-estados/sc.html - Dados de Santa Catarina
- https://www.ibge.gov.br/cidades-e-estados/pe.html - Dados de Pernambuco
- https://www.unimedfortaleza.com.br/blog/alimentacao/alimentos-vitamina-d - Dados da Unimed sobre vitamina D
- https://www.globaldietarydatabase.org/ - Base de hábitos alimentares da Global Dietary
- https://brasil.io/dataset/covid19/caso/ - Base do COVID19 do Brasil.IO